In [ ]:
!pip install transformers -U

In [6]:
import gc
import torch

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [1]:
import transformers
import torch

model_id = "OpenLLM-Ro/RoLlama3-8b-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
# messages = [
#     {"role": "system", "content": 'You are a helpful asisstant that classifies Romanian meme captions as political or not. You briefly explain your reasoning and then output "YES" or "NO".'},
#     {"role": "user", "content": "Ce aveti fără zahăr și să nu îngrașe? Șervețele!"},
# ]


terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

messages = [
    #{"role": "system", "content": 'Ești un asistent util care decide dacă conținutul text al unui "meme" (o glumă virtuală) are conotații politice sau nu. Întâi, îți explici într-o propoziție raționamentul, iar apoi răspunzi la final cu "DA" sau "NU".'},
    {"role": "system", "content": 'Ești un asistent util care clasifică conținutul unor "meme"-uri drept politic sau nu. Vei primi doar partea textul. Vei explica pe scurt decizia, iar apoi vei răspunde la final cu "DA" sau "NU".'},
    {"role": "user", "content": """MASCA ESTE OBLIGATORIE! \nROMÂNII:"""}
]

outputs = pipeline(
    messages,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': 'Textul dat este legat de un meme, care este un tip de conținut social media care poate fi clasificat ca non-politic. În acest caz, meme-ul este despre o mască de față, care este o practică de siguranță în timpul pandemiei de COVID-19, și nu implică niciun subiect politic. Deci, răspunsul este „NU”.'}


In [43]:
import pandas as pd
df = pd.read_csv("/gd/MyDrive/romemes/political_texts.csv")

In [44]:
texts = df['text'].tolist()

In [45]:
from tqdm import tqdm

In [54]:
answers = []
for text in tqdm(texts):
  messages = [
      #{"role": "system", "content": 'Ești un asistent util care decide dacă conținutul text al unui "meme" (o glumă virtuală) are conotații politice sau nu. Întâi, îți explici într-o propoziție raționamentul, iar apoi răspunzi la final cu "DA" sau "NU".'},
      {"role": "system", "content": 'Ești un asistent util care clasifică conținutul unui meme drept politic sau nu. Un meme este o glumă virtuală formată dintr-o imagine și un text. Tu vei primi doar textul. Este obligatoriu să explici textul meme-ului și de ce acesta ar fi politic sau nu, iar la finalul mesajului tău vei răspunde cu "DA" sau "NU", '},
      {"role": "user", "content": text}
  ]

  outputs = pipeline(
      messages,
      max_new_tokens=512,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  answers.append(outputs[0]['generated_text'][-1])

100%|██████████| 462/462 [33:42<00:00,  4.38s/it]


In [57]:
ans = []
for output in answers:
  ans.append(output['content'])

In [60]:
df.to_csv("/gd/MyDrive/romemes/rollama.csv")

In [58]:
ans

['Textul meme-ului pare să facă o glumă despre bananele care au pete întunecate din cauza implicării lor în lupte ilegale pe timp de noapte. Această glumă se referă la o situație fantezistă și nu la un subiect politic. Prin urmare, răspunsul este „NU”.',
 'Textul meme-ului este „MASCA ESTE OBLIGATORIE! ROMÂNII:” și pare să se refere la un eveniment sau o situație în care oamenii sunt obligați să poarte măști în timpul pandemiei de COVID-19. Deși această declarație nu este neapărat politică, poate fi considerată politică dacă este legată de un meme care discută politicile guvernamentale, cum ar fi măsurile de sănătate publică sau reglementările legate de pandemie. În acest caz, nu este clar dacă meme-ul este politic sau nu, deci răspunsul meu este „NU”.',
 'Acest meme este despre o situație personală și nu are conținut politic. Răspuns: NU',
 'Textul meme-ului pare să discute despre o știre legată de producția de vaccinuri antigripale în cadrul unei instituții medicale. Deși acest subie

In [59]:
df['ans'] = ans

In [61]:
df

,Unnamed: 0,ID,Political,text,ans
0,0,100001,Yes,ȘTII CĂ BANANELE AU ACELE PETE NEGRE PENTRU CĂ...,Textul meme-ului pare să facă o glumă despre b...
1,1,100002,Yes,MASCA ESTE OBLIGATORIE! \nROMÂNII:,Textul meme-ului este „MASCA ESTE OBLIGATORIE!...
2,2,100003,No,Când vrei să te răzbuni pe musca care nu te-a ...,Acest meme este despre o situație personală și...
3,3,100004,Yes,Mai multe filiale ale Institului Cantacuzino a...,Textul meme-ului pare să discute despre o știr...
4,4,100005,No,Ce aveti fără zahăr și să nu îngrașe? Șervețele!,Textul meme-ului este o glumă despre șervețele...
...,...,...,...,...,...
457,457,500095,No,Bucurestean in hainele de casa pe timpul iernii,NU
458,458,500096,No,Când încerc sa mănânc noaptea în liniște și sc...,Acest meme este o glumă despre o persoană care...
459,459,500097,No,Ce animal! Nu te speria!Îl țin cu două mâini!,NU
460,460,500098,No,Intoarcerea la serviciu dupa un weekend prelun...,Textul meme-ului descrie o situație umoristică...


In [3]:
# connecting to google drive
from google.colab import drive
import os
if not os.path.exists('/gd'):
  drive.mount('/gd')

Mounted at /gd
